# Part 12

# Using File Search

Universal code for the entire notebook

In [ ]:
# Uncomment the line below to make sure you have all the packages needed
# %pip install -r requirements.txt

In [2]:
# Import necessary libraries
from openai import OpenAI  # Used for interacting with OpenAI's API
from typing_extensions import override  # Used for overriding methods in subclasses
from openai import AssistantEventHandler  # Used for handling events related to OpenAI assistants

import re # Used for regular expressions


In [3]:
# Create an instance of the OpenAI class to interact with the API.
# This assumes you have set the OPENAI_API_KEY environment variable.
client = OpenAI() 

In [26]:
# Event handler class to handle events related to streaming output from the assistant
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nASSISTANT MESSAGE >\n", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nASSISTANT MESSAGE >\n{tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

In [27]:
# upload files for later use
alice_file = client.files.create(file=open("./artifacts/Alice_in_Wonderland.pdf","rb"), purpose="assistants")
oz_file = client.files.create(file=open("./artifacts/The_Wonderful_Wizard_of_Oz.txt","rb"), purpose="assistants")
dracula_file = client.files.create(file=open("./artifacts/Dracula.pdf","rb"), purpose="assistants")
frank_file = client.files.create(file=open("./artifacts/Frankenstein.pdf","rb"), purpose="assistants")


In [28]:
print(alice_file.filename)
print(alice_file.id)
print("\n")
print(oz_file.filename)
print(oz_file.id)
print("\n")
print(dracula_file.filename)
print(dracula_file.id)
print("\n")
print(frank_file.filename)
print(frank_file.id)
print("\n")

Alice_in_Wonderland.pdf
file-0FIEIREs7FBx2I0JWTCvX6lr


The_Wonderful_Wizard_of_Oz.txt
file-G9SKXdqujcOdgPLxZVsIYvyH


Dracula.pdf
file-2Pt5p1we3eN6PbnVGfkHrX4W


Frankenstein.pdf
file-qvUMg5D47GSGgxieOHPPswx1




### Creating an Assistant with File Search enabled

Our first step is to create an Assistant that can do file searching regardless of where the vector store resides (Assistant or Thread)

In [29]:
# Create an assistant using the client library.
assistant = client.beta.assistants.create(
    model="gpt-4o",  # Specify the model to be used.
    
    instructions=""" 
        You are a helpful assistant that answers questions about the stories in your files. The stories are from a variety of authors. 
        You will answer questions from the user about the stories. All you will do is answer questions about the stories in the files and provide related information.
        If the user asks you a question that is not related to the stories in the files, you should let them know that you can only answer questions about the stories.
    """,
    
    name="File Search Demo Assistant - Stories",  # Give the assistant a name.
    
    tools=[{"type": "file_search"}], # Add the file search capability to the assistant.
    
    metadata={  # Add metadata about the assistant's capabilities.
        "can_be_used_for_file_search": "True",
        "can_hold_vector_store": "True",
    },
    temperature=1,  # Set the temperature for response variability.
    top_p=1,  # Set the top_p for nucleus sampling.
)

# Print the details of the created assistant to check its properties.
print(assistant)  # Print the full assistant object.
print("\n\n")
print(assistant.name)  # Print the name of the assistant.
print(assistant.metadata)  # Print the metadata of the assistant.

Assistant(id='asst_DoFXrEezZFG3QxW5mhvvtTFp', created_at=1717674131, description=None, instructions=' \n        You are a helpful assistant that answers questions about the stories in your files. The stories are from a variety of authors. \n        You will answer questions from the user about the stories. All you will do is answer questions about the stories in the files and provide related information.\n        If the user asks you a question that is not related to the stories in the files, you should let them know that you can only answer questions about the stories.\n    ', metadata={'can_be_used_for_file_search': 'True', 'can_hold_vector_store': 'True'}, model='gpt-4o', name='File Search Demo Assistant - Stories', object='assistant', tools=[FileSearchTool(type='file_search', file_search=None)], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)



File Search Demo Assist

## Creating a Vector Store

Now we will create our vector store to hold our files and add files at the same time.

In [30]:
from contextlib import ExitStack

# Create a vector store with a name for the store.
vector_store = client.beta.vector_stores.create(name="Great Fiction Stories")

# Ready the files for upload to the vector store.
file_paths = ["./artifacts/I_Am_Legend.pdf", "./artifacts/The_Veldt.pdf"]

# Using ExitStack to manage multiple context managers and ensure they are properly closed.
with ExitStack() as stack:
    # Open each file in binary read mode and add the file stream to the list
    file_streams = [stack.enter_context(open(path, "rb")) for path in file_paths]

    # Use the upload and poll helper method to upload the files, add them to the vector store,
    # and poll the status of the file batch for completion.
    file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id, files=file_streams
    )

    # Print the vector store information
    print(vector_store.name)
    print(vector_store.id)
    
    # Print the status and the file counts of the batch to see the results
    print(file_batch.status)
    print(file_batch.file_counts)


Great Fiction Stories
vs_4VB1NSiJioJqeA74T9DQAKJJ
completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


# Part 13

### Attaching the Vector Store to the Assistant

We have an Assistant that has File Search enabled and we have a Vector Store with files in them. It's time to join the two up. 

In [31]:
try:
    # Attach the vector store to the assistant to enable file search capabilities.
    assistant = client.beta.assistants.update(
        assistant_id=assistant.id,
        tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    )

    # Print the assistant's tools and tool resources to verify the attachment of the vector store.
    print("Assistant Tools:")
    for tool in assistant.tools:
        print(f" - {tool}")

    # Print the assistant's tool resources to verify the attachment of the vector store
    print("\nAssistant Tool Resources:")
    for resource, details in assistant.tool_resources:
        print(f" - {resource}: {details}")

except Exception as e:
    print(f"An error occurred while updating the assistant: {e}")


Assistant Tools:
 - FileSearchTool(type='file_search', file_search=None)

Assistant Tool Resources:
 - code_interpreter: None
 - file_search: ToolResourcesFileSearch(vector_store_ids=['vs_4VB1NSiJioJqeA74T9DQAKJJ'])


### Creating an Assistant and Vector Store at the Same Time

If we have file id's we can just feed them in when creating an assistant to get the Assistant and the Vector Store at the same time using the vector_stores option.

In [32]:

# Create an assistant using the client library.
try:
    assistant_with_vector_store = client.beta.assistants.create(
        model="gpt-4o",  # Specify the model to be used.
        instructions=(
            "You are a helpful assistant that answers questions about the stories in your files. "
            "The stories are from a variety of authors. "
            "You will answer questions from the user about the stories. All you will do is answer questions about the stories in the files and provide related information. "
            "If the user asks you a question that is not related to the stories in the files, you should let them know that you can only answer questions about the stories."
        ),
        name="Quick Assistant and Vector Store at Once",  # Give the assistant a name.
        tools=[{"type": "file_search"}],  # Add the file search capability to the assistant.
        # Create a vector store and attach it to the assistant in one step.
        tool_resources={
            "file_search": {
                "vector_stores": [
                    {
                        "name": "Vector Store Auto Attached to Assistant",
                        "file_ids": [
                            oz_file.id,
                            alice_file.id
                        ],
                        "metadata": {
                            "Book1": "Wizard of Oz", 
                            "Book2": "Alice in Wonderland"
                        }
                    }
                ]
            }
        },
        metadata={  # Add metadata about the assistant's capabilities.
            "can_be_used_for_file_search": "True",
            "has_vector_store": "True",
        },
        temperature=1,  # Set the temperature for response variability.
        top_p=1,  # Set the top_p for nucleus sampling.
    )
except Exception as e:
    print(f"An error occurred while creating the assistant: {e}")
else:
    # Print the details of the created assistant to check its properties.
    print(assistant_with_vector_store)  # Print the full assistant object.
    print("\n\n")
    print("Assistant Name: " + assistant_with_vector_store.name)  # Print the name of the assistant.
    print("\n")
    
    # get the vector store information
    unnamed_assistant_vector_store = client.beta.vector_stores.retrieve(assistant_with_vector_store.tool_resources.file_search.vector_store_ids[0])
    print("Vector Store Name: " + str(unnamed_assistant_vector_store.name))
    print("Vector Store Id: " + unnamed_assistant_vector_store.id)
    print("Vector Store Metadata: " + str(unnamed_assistant_vector_store.metadata))


Assistant(id='asst_Xqe7RKL94sA7oI3jQcI47Qor', created_at=1717687018, description=None, instructions='You are a helpful assistant that answers questions about the stories in your files. The stories are from a variety of authors. You will answer questions from the user about the stories. All you will do is answer questions about the stories in the files and provide related information. If the user asks you a question that is not related to the stories in the files, you should let them know that you can only answer questions about the stories.', metadata={'can_be_used_for_file_search': 'True', 'has_vector_store': 'True'}, model='gpt-4o', name='Quick Assistant and Vector Store at Once', object='assistant', tools=[FileSearchTool(type='file_search', file_search=None)], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_gG4yDnQuxf5JwD6jPwilnPDz'])), top_p=1.0)



Assistant Name: Quick Assistant

## Creating Threads with Vector Stores

### Creating Vector Stores with Thread Messages

You can create a vector store in threads with one of two ways: messages or during thread creation. If you create a thread with a vector store then it will also be searched when looking for information during the run. First, let's take a look at the most common scenario, vector stores created with messages in the thread.

In [33]:
# We assume that the user has given us a file to upload
message_file = client.files.create(
    file=open("./artifacts/War_of_the_Worlds.txt", "rb"), purpose="assistants"
)

# Create a thread and attach the file to the message
thread_with_file_attachment = client.beta.threads.create(
    messages=[
    {
    "role": "user",
    "content": "List all the books you have access to in your files.",
    
    # Attach the new file to the message.
    "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
    ],
    }
]
)

# Give the vector store a friendly name for easy reference.
updated_vector_store = client.beta.vector_stores.update(
    vector_store_id=thread_with_file_attachment.tool_resources.file_search.vector_store_ids[0],
    name="Vector Store Auto Attached to Thread When Message File Attached",
)

# The thread now has a vector store with that file in its tool resources.
print(updated_vector_store.name)
print(updated_vector_store.id)

# Alternate way to get the vector store id from the thread
print(thread_with_file_attachment.tool_resources.file_search.vector_store_ids[0])   



Vector Store Auto Attached to Thread When Message File Attached
vs_W0nvRHVEeomtdrpL2PMh1Pne
vs_W0nvRHVEeomtdrpL2PMh1Pne


### Creating Vector Stores at Thread Creation Time

Now, let's look at creating a vector store when we create our thread. 

In [42]:
thread_with_vector_store = client.beta.threads.create(
    messages=[
    {
    "role": "user",
    "content": "List every book you have access to. Just give me the list and nothing else. Then tell me who the main character was in each book",
    }
    ],
    tool_resources={
            "file_search": {
                "vector_stores": [
                    {
                        "name": "Vector Store Auto Attached to Thread",
                        "file_ids": [
                            dracula_file.id,
                            frank_file.id
                        ],
                        "metadata": {
                            "Book1": "Dracula", 
                            "Book2": "Frankenstein"
                        }
                    }
                ]
            }
        },
        metadata={  # Add metadata about the assistant's capabilities.
            "can_be_used_for_file_search": "True",
            "has_vector_store": "True",
        },
)

# Print the details of the created thread to check its properties
print(thread_with_vector_store)  # Print the full thread object.
print("\n")
print("Thread ID: " + thread_with_vector_store.id)  # Print the ID of the thread.
print("Thread Metadata: " + str(thread_with_vector_store.metadata))  # Print the metadata of the thread.
print("Thread Tool Resources: " + str(thread_with_vector_store.tool_resources))  # Print the tool resources of the thread.

Thread(id='thread_F0tueXawpqN9BAowCaD8Y3mr', created_at=1717688420, metadata={'can_be_used_for_file_search': 'True', 'has_vector_store': 'True'}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_BczptHYs6oRzTNYmeHsjFDIo'])))


Thread ID: thread_F0tueXawpqN9BAowCaD8Y3mr
Thread Metadata: {'can_be_used_for_file_search': 'True', 'has_vector_store': 'True'}
Thread Tool Resources: ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_BczptHYs6oRzTNYmeHsjFDIo']))


## Getting the Results

We have built up Assistants and Threads with vector stores and now we want to use them. We need to set up a run to make this happen. Recall there are two approaches: streaming or non-streaming. We will do both.

### Streaming Run

First, let's do the, more common, streaming run to get our output.

In [43]:
# Using our first assistant
with client.beta.threads.runs.stream(
    thread_id=thread_with_vector_store.id,
    assistant_id=assistant.id,
    instructions="""
    You are a helpful assistant. 
    ALWAYS read all the files you have before answering questions about them. 
    Only answer the user's question about the information you have in your files.
    """,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


ASSISTANT MESSAGE >
file_search


ASSISTANT MESSAGE >
### Book List:
1. Frankenstein by Mary Wollstonecraft Shelley
2. Dracula by Bram Stoker

### Main Characters:
1. **Frankenstein**:
    - Main Character: Victor Frankenstein[0]
2. **Dracula**:
    - Main Characters: Jonathan Harker, Mina Murray, and Count Dracula[1]
[0] Frankenstein.pdf
[1] Dracula.pdf


## Run Without Streaming

Now let's do a run without streaming to compare the two.

In [36]:
# Use our second assistant
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread_with_file_attachment.id, assistant_id=assistant_with_vector_store.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread_with_file_attachment.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

Here are the book titles I have access to from your files:

1. *Alice's Adventures in Wonderland* by Lewis Carroll
2. *Through the Looking-Glass, and What Alice Found There* by Lewis Carroll
3. *The Wonderful Wizard of Oz* by L. Frank Baum
4. *The War of the Worlds* by H. G. Wells   .



# Part 14

## Creating a Simple Vector Store

Let's create the bare minimum vector store. 

In [4]:
# Absolute minimum example of creating a vector store
vector_store = client.beta.vector_stores.create(
)
print(vector_store)


VectorStore(id='vs_W1dbdzPZ2bxbVMp5DlSVw3XH', created_at=1717702882, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1717702882, metadata={}, name=None, object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)


### Setting the Expiration

If your vector store has a time limit, you can set and expiration for it. Vector stores created with threads or messages have a default expiration of 7 days by default. Normal vector stores do not have an expiration by default. 

In [5]:
# Setting a name and expiration time for the vector store
vector_store = client.beta.vector_stores.create(
    name="Dead Vector Store Walking",
    expires_after={
        "anchor": "last_active_at",
        "days": 10
    }
)
print(vector_store)


VectorStore(id='vs_xMm4BzNCxOmyZFzQzwTv8qVV', created_at=1717703026, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1717703026, metadata={}, name='Dead Vector Store Walking', object='vector_store', status='completed', usage_bytes=0, expires_after=ExpiresAfter(anchor='last_active_at', days=10), expires_at=1718567026)


### Chunking Strategy

Time to see how we would implement a chunking strategy without the Auto turned on. In this case, we set "type" to "static" and then indicate the parameters for chunk size and overlap.

In [6]:
# Create a vector store with static chunking strategy
vector_store = client.beta.vector_stores.create(
    chunking_strategy={
        "type": "static",
        "static": {
            "max_chunk_size_tokens": 1000,  # Customize these values as needed
            "chunk_overlap_tokens": 500    # Customize these values as needed
        }
    }
)
print(vector_store)

VectorStore(id='vs_TJIE828UbugAfJvpiC38SBru', created_at=1717703407, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1717703407, metadata={}, name=None, object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)


## Listing Vector Stores

It's important to be able to list our vector stores so we can work with them in code.

In [7]:
# Get a list of our vector stores up to the limit
# The default page limit is 100, but we can specify a different limit
# Use "after" to go through more than 100 vector stores
vector_stores = client.beta.vector_stores.list()

# Convert the SyncCursorPage to a list to get the number of 
# vector stores in the current page
vector_store_list = list(vector_stores.data)

# Show the number of vector stores in the current page
print("Vector Stores: " + str(len(vector_store_list)))

# Loop through the vector stores in the current page
# and print their names and IDs
for vector_store in vector_store_list:
    print("Name: " + str(vector_store.name))
    print("ID: " + str(vector_store.id))
    print("\n")


Vector Stores: 10
Name: None
ID: vs_TJIE828UbugAfJvpiC38SBru


Name: Dead Vector Store Walking
ID: vs_xMm4BzNCxOmyZFzQzwTv8qVV


Name: None
ID: vs_W1dbdzPZ2bxbVMp5DlSVw3XH


Name: Vector Store Auto Attached to Thread
ID: vs_BczptHYs6oRzTNYmeHsjFDIo


Name: Vector Store Auto Attached to Thread
ID: vs_OH3COfzRDcLMKKLRuDUK1mj0


Name: Vector Store Auto Attached to Thread
ID: vs_FiJTdq7OSNBdp5UOgWCJoIsE


Name: Vector Store Auto Attached to Thread
ID: vs_Cj2J7ZDAgyxrc213nN58pjyJ


Name: Vector Store Auto Attached to Thread When Message File Attached
ID: vs_W0nvRHVEeomtdrpL2PMh1Pne


Name: Vector Store Auto Attached to Assistant
ID: vs_gG4yDnQuxf5JwD6jPwilnPDz


Name: Great Fiction Stories
ID: vs_4VB1NSiJioJqeA74T9DQAKJJ




## Retrieving Vector Stores

Given any vector store id, we can retrieve a vector store. Let's get creative and get a vector store id using it's name and regular expressions. This assumes you have done the prior cells in this notebook and now want to get the vector store id for a particular vector store with a name. 

In [8]:
# Since we retrieved a list of vector stores in the previous example, let's leverage that list to find a specific vector store
# We will look for a vector store with a specific name, like "Fiction",  using a regular expression and print its ID


# Filter the list to find stores with names that match the pattern "Fiction" (case insensitive), ignoring NoneType names
list_of_fiction_stores = [store for store in vector_store_list if store.name is not None and re.search("[Ff]iction", store.name)]

# Check to see if we found any stores
if list_of_fiction_stores:
    fiction_store = list_of_fiction_stores[0]
    print("Fiction Store ID: " + fiction_store.id)
    
    # Now let's retrieve the vector store using the ID of the fiction store found
    retrieved_fiction_vector_store = client.beta.vector_stores.retrieve(vector_store_id=fiction_store.id)
    
    # Print the name and id of the retrieved vector store
    if retrieved_fiction_vector_store.name is not None:
        print("Retrieved Fiction Store Name: " + retrieved_fiction_vector_store.name)
    else:
        print("Retrieved Fiction Store has no name")
        
    print("Retrieved Fiction Store ID: " + retrieved_fiction_vector_store.id)
else:    
    print("No Fiction Store Found")





Fiction Store ID: vs_4VB1NSiJioJqeA74T9DQAKJJ
Retrieved Fiction Store Name: Great Fiction Stories
Retrieved Fiction Store ID: vs_4VB1NSiJioJqeA74T9DQAKJJ


## Updating Vector Stores

Now that we have retrieved the vector store that we wanted, let's modify it.

In [9]:
# Show the vector store information
print("Vector Store Name: " + retrieved_fiction_vector_store.name)
print("Vector Store ID: " + retrieved_fiction_vector_store.id)
print("Vector Store Metadata: " + str(retrieved_fiction_vector_store.metadata))
print("Vector Store Expires After: " + str(retrieved_fiction_vector_store.expires_after))
print("\n")

# Update the vector store with an expiration and some new metadata
updated_fiction_vector_store = client.beta.vector_stores.update(
    vector_store_id=retrieved_fiction_vector_store.id,
    expires_after={
        "anchor": "last_active_at",
        "days": 5
    },
    metadata={
        "Genre": "Science Fiction",
        "Book1": "I Am Legend",
        "Book2": "The Veldt"
    }
)

# Show the updated vector store information
print("Updated Vector Store Name: " + updated_fiction_vector_store.name)
print("Updated Vector Store ID: " + updated_fiction_vector_store.id)
print("Updated Vector Store Metadata: " + str(updated_fiction_vector_store.metadata))
print("Updated Vector Store Expires After: " + str(updated_fiction_vector_store.expires_after))

Vector Store Name: Great Fiction Stories
Vector Store ID: vs_4VB1NSiJioJqeA74T9DQAKJJ
Vector Store Metadata: {}
Vector Store Expires After: None


Updated Vector Store Name: Great Fiction Stories
Updated Vector Store ID: vs_4VB1NSiJioJqeA74T9DQAKJJ
Updated Vector Store Metadata: {'Genre': 'Science Fiction', 'Book1': 'I Am Legend', 'Book2': 'The Veldt'}
Updated Vector Store Expires After: ExpiresAfter(anchor='last_active_at', days=5)


## Deleting Vector Store

Inevitably we will want to get rid of vector stores to keep our environment clean. We have decided to get rid of any Assistant with the word "Attached" in it. 

In [11]:
# Get a list of our vector stores up to the limit
# The default page limit is 100, but we can specify a different limit
# Use "after" to go through more than 100 vector stores
more_vector_stores = client.beta.vector_stores.list()

# Convert the SyncCursorPage to a list to get the number of 
# vector stores in the current page
more_vector_store_list = list(vector_stores.data)

# Show the number of vector stores in the current page
print("Vector Stores: " + str(len(vector_store_list)))

# Loop through the vector stores in the current page
# and print their names and IDs
for vector_store in more_vector_store_list:
    print("Name: " + str(vector_store.name))
    print("ID: " + str(vector_store.id))
    print("\n")

Vector Stores: 10
Name: None
ID: vs_TJIE828UbugAfJvpiC38SBru


Name: Dead Vector Store Walking
ID: vs_xMm4BzNCxOmyZFzQzwTv8qVV


Name: None
ID: vs_W1dbdzPZ2bxbVMp5DlSVw3XH


Name: Vector Store Auto Attached to Thread
ID: vs_BczptHYs6oRzTNYmeHsjFDIo


Name: Vector Store Auto Attached to Thread
ID: vs_OH3COfzRDcLMKKLRuDUK1mj0


Name: Vector Store Auto Attached to Thread
ID: vs_FiJTdq7OSNBdp5UOgWCJoIsE


Name: Vector Store Auto Attached to Thread
ID: vs_Cj2J7ZDAgyxrc213nN58pjyJ


Name: Vector Store Auto Attached to Thread When Message File Attached
ID: vs_W0nvRHVEeomtdrpL2PMh1Pne


Name: Vector Store Auto Attached to Assistant
ID: vs_gG4yDnQuxf5JwD6jPwilnPDz


Name: Great Fiction Stories
ID: vs_4VB1NSiJioJqeA74T9DQAKJJ




In [12]:
# Since we retrieved a list of vector stores in the previous example, let's leverage that list to find a specific vector store
# We will look for a vector store with a specific name, like "Fiction",  using a regular expression and print its ID


# Filter the list to find stores with names that match the pattern "Fiction" (case insensitive), ignoring NoneType names
list_of_attached_stores = [store for store in more_vector_store_list if store.name is not None and re.search("[Aa]ttached", store.name)]

# check to see if we have any stores then show all the attached stores

for attached_store in list_of_attached_stores:
    
    # Now let's retrieve the vector store using the ID of the attached store found
    retrieved_attached_vector_store = client.beta.vector_stores.retrieve(vector_store_id=attached_store.id)
    
    # Print the name and id of the retrieved vector store
    if retrieved_attached_vector_store.name is not None:
        print("Retrieved Attached Store Name: " + retrieved_attached_vector_store.name)
    else:
        print("Retrieved Attached Store has no name")
        
    print("Retrieved Attached Store ID: " + retrieved_attached_vector_store.id)
    print("Retrieved Attached Store Metadata: " + str(retrieved_attached_vector_store.metadata))
    print("Retrieved Attached Store Expires After: " + str(retrieved_attached_vector_store.expires_after))
    print("\n")
    
    


Retrieved Attached Store Name: Vector Store Auto Attached to Thread
Retrieved Attached Store ID: vs_BczptHYs6oRzTNYmeHsjFDIo
Retrieved Attached Store Metadata: {'Book1': 'Dracula', 'Book2': 'Frankenstein'}
Retrieved Attached Store Expires After: ExpiresAfter(anchor='last_active_at', days=7)


Retrieved Attached Store Name: Vector Store Auto Attached to Thread
Retrieved Attached Store ID: vs_OH3COfzRDcLMKKLRuDUK1mj0
Retrieved Attached Store Metadata: {'Book1': 'Dracula', 'Book2': 'Frankenstein'}
Retrieved Attached Store Expires After: ExpiresAfter(anchor='last_active_at', days=7)


Retrieved Attached Store Name: Vector Store Auto Attached to Thread
Retrieved Attached Store ID: vs_FiJTdq7OSNBdp5UOgWCJoIsE
Retrieved Attached Store Metadata: {'Book1': 'Dracula', 'Book2': 'Frankenstein'}
Retrieved Attached Store Expires After: ExpiresAfter(anchor='last_active_at', days=7)


Retrieved Attached Store Name: Vector Store Auto Attached to Thread
Retrieved Attached Store ID: vs_Cj2J7ZDAgyxrc213nN

In [13]:
# Now let's delete the vector stores in our list of retrieved vector stores
for vector_store in list_of_attached_stores:
    try:
        # Delete the vector store
        client.beta.vector_stores.delete(vector_store_id=vector_store.id)
        print(f"Deleted vector store with Name: {vector_store.name}" 
                + f"\nDeleted vector store with ID: {vector_store.id}\n\n")
    except Exception as e:
        print(f"An error occurred while deleting the vector store: {e}")




Deleted vector store with Name: Vector Store Auto Attached to Thread
Deleted vector store with ID: vs_BczptHYs6oRzTNYmeHsjFDIo


Deleted vector store with Name: Vector Store Auto Attached to Thread
Deleted vector store with ID: vs_OH3COfzRDcLMKKLRuDUK1mj0


Deleted vector store with Name: Vector Store Auto Attached to Thread
Deleted vector store with ID: vs_FiJTdq7OSNBdp5UOgWCJoIsE


Deleted vector store with Name: Vector Store Auto Attached to Thread
Deleted vector store with ID: vs_Cj2J7ZDAgyxrc213nN58pjyJ


Deleted vector store with Name: Vector Store Auto Attached to Thread When Message File Attached
Deleted vector store with ID: vs_W0nvRHVEeomtdrpL2PMh1Pne


Deleted vector store with Name: Vector Store Auto Attached to Assistant
Deleted vector store with ID: vs_gG4yDnQuxf5JwD6jPwilnPDz


